

---

# **Chapter 35: Custom Widgets & Painters**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Create composite custom widgets by combining existing primitives
- Implement CustomPainter for complex 2D graphics and data visualization
- Design CustomClipper for custom shapes and clipping paths
- Apply gradient shaders and visual effects using Paint configurations
- Build implicitly animated widgets for smooth property transitions
- Understand when to use composition vs. custom render objects

---

## **Prerequisites**

- Completed Chapter 34: Rendering Optimization (understanding of Paint phase, canvas)
- Completed Chapter 7: Widget Deep Dive (understanding of BuildContext, Keys)
- Understanding of Dart's math library for geometry calculations
- Familiarity with Flutter's BoxConstraints and layout protocol

---

## **35.1 Creating Composite Custom Widgets**

Before diving into low-level rendering, most custom UI needs can be met by composing existing widgets creatively. This is the preferred approach in Flutter.

### **Compositional Widget Design**

```dart
// File: lib/custom_widgets/composite_widgets.dart
import 'package:flutter/material.dart';

// Pattern 1: Compound Widget - Multiple widgets working together
class GradientCard extends StatelessWidget {
  final Widget child;
  final List<Color> gradientColors;
  final double elevation;
  final BorderRadius borderRadius;
  
  const GradientCard({
    Key? key,
    required this.child,
    required this.gradientColors,
    this.elevation = 4.0,
    this.borderRadius = const BorderRadius.all(Radius.circular(12)),
  }) : super(key: key);
  
  @override
  Widget build(BuildContext context) {
    // Composing Container, BoxDecoration, Material, and ClipRRect
    // to create a reusable, complex visual element
    
    return Material(
      // Material provides elevation (shadow) and ink effects
      elevation: elevation,
      borderRadius: borderRadius,
      color: Colors.transparent, // Let gradient show through
      
      child: ClipRRect(
        // Clipping ensures child content respects border radius
        borderRadius: borderRadius,
        
        child: Container(
          decoration: BoxDecoration(
            // Linear gradient background
            gradient: LinearGradient(
              begin: Alignment.topLeft,
              end: Alignment.bottomRight,
              colors: gradientColors,
              // Smooth color transitions
              stops: const [0.0, 0.5, 1.0],
            ),
          ),
          
          // Padding inside the card
          padding: const EdgeInsets.all(16.0),
          
          // The actual content passed by parent
          child: child,
        ),
      ),
    );
  }
}

// Usage
class GradientCardExample extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return GradientCard(
      gradientColors: [Colors.purple, Colors.blue, Colors.cyan],
      elevation: 8.0,
      child: Column(
        crossAxisAlignment: CrossAxisAlignment.start,
        children: [
          Text(
            'Premium Feature',
            style: TextStyle(
              color: Colors.white,
              fontSize: 24,
              fontWeight: FontWeight.bold,
            ),
          ),
          SizedBox(height: 8),
          Text(
            'Unlock advanced capabilities with our pro plan.',
            style: TextStyle(color: Colors.white70),
          ),
        ],
      ),
    );
  }
}

// Pattern 2: Smart Widget with internal state management
class ExpandableSection extends StatefulWidget {
  final String title;
  final Widget child;
  final Duration animationDuration;
  
  const ExpandableSection({
    Key? key,
    required this.title,
    required this.child,
    this.animationDuration = const Duration(milliseconds: 300),
  }) : super(key: key);
  
  @override
  _ExpandableSectionState createState() => _ExpandableSectionState();
}

class _ExpandableSectionState extends State<ExpandableSection> 
    with SingleTickerProviderStateMixin {
  bool _isExpanded = false;
  late AnimationController _controller;
  late Animation<double> _heightFactor;
  
  @override
  void initState() {
    super.initState();
    _controller = AnimationController(
      duration: widget.animationDuration,
      vsync: this,
    );
    
    _heightFactor = Tween<double>(begin: 0.0, end: 1.0).animate(
      CurvedAnimation(
        parent: _controller,
        curve: Curves.easeInOut,
      ),
    );
  }
  
  void _toggleExpansion() {
    setState(() {
      _isExpanded = !_isExpanded;
      if (_isExpanded) {
        _controller.forward();
      } else {
        _controller.reverse();
      }
    });
  }
  
  @override
  Widget build(BuildContext context) {
    return Column(
      children: [
        // Header row with tap handling
        InkWell(
          onTap: _toggleExpansion,
          child: Padding(
            padding: EdgeInsets.all(16.0),
            child: Row(
              mainAxisAlignment: MainAxisAlignment.spaceBetween,
              children: [
                Text(
                  widget.title,
                  style: Theme.of(context).textTheme.titleMedium,
                ),
                // Animated rotation for arrow icon
                AnimatedRotation(
                  turns: _isExpanded ? 0.5 : 0.0,
                  duration: widget.animationDuration,
                  child: Icon(Icons.expand_more),
                ),
              ],
            ),
          ),
        ),
        
        // Animated content area
        AnimatedBuilder(
          animation: _controller,
          builder: (context, child) {
            return ClipRect(
              // Clip prevents overflow during animation
              child: Align(
                heightFactor: _heightFactor.value,
                alignment: Alignment.topCenter,
                child: child,
              ),
            );
          },
          child: Padding(
            padding: EdgeInsets.symmetric(horizontal: 16.0),
            child: widget.child,
          ),
        ),
        
        // Divider when expanded
        AnimatedOpacity(
          opacity: _isExpanded ? 1.0 : 0.0,
          duration: widget.animationDuration,
          child: Divider(),
        ),
      ],
    );
  }
  
  @override
  void dispose() {
    _controller.dispose();
    super.dispose();
  }
}

// Pattern 3: Render Object Wrapper - Accessing size and position
class MeasurableWidget extends SingleChildRenderObjectWidget {
  final void Function(Size size) onMeasure;
  
  const MeasurableWidget({
    Key? key,
    required Widget child,
    required this.onMeasure,
  }) : super(key: key, child: child);
  
  @override
  RenderObject createRenderObject(BuildContext context) {
    return _MeasurableRenderObject(onMeasure);
  }
  
  @override
  void updateRenderObject(
    BuildContext context,
    _MeasurableRenderObject renderObject,
  ) {
    renderObject.onMeasure = onMeasure;
  }
}

class _MeasurableRenderObject extends RenderProxyBox {
  void Function(Size size) onMeasure;
  
  _MeasurableRenderObject(this.onMeasure);
  
  @override
  void performLayout() {
    super.performLayout();
    
    // Notify parent of final size after layout
    if (size.isFinite) {
      onMeasure(size);
    }
  }
}

// Usage - Getting child size for parent decisions
class ResponsiveLayout extends StatefulWidget {
  @override
  _ResponsiveLayoutState createState() => _ResponsiveLayoutState();
}

class _ResponsiveLayoutState extends State<ResponsiveLayout> {
  Size? _childSize;
  
  @override
  Widget build(BuildContext context) {
    return MeasurableWidget(
      onMeasure: (size) {
        // Update state with measured size
        // This triggers rebuild but demonstrates pattern
        if (_childSize != size) {
          setState(() => _childSize = size);
        }
      },
      child: Container(
        width: double.infinity,
        child: Text('Measure me'),
      ),
    );
  }
}
```

**Explanation:**

- **Compositional approach**: `GradientCard` combines `Material` (shadows), `ClipRRect` (rounded corners), `Container` (decoration), and `BoxDecoration` (gradients) without any custom painting. This leverages Flutter's optimized render objects.
- **Stateful compound widgets**: `ExpandableSection` encapsulates animation logic, state management, and visual structure into a reusable package. Parents simply provide content; the widget handles expansion/collapse animations.
- **RenderObject wrappers**: When you need size information or layout behavior that widgets can't provide, wrap a `RenderProxyBox`. `MeasurableWidget` demonstrates intercepting layout to report size upward.
- **Performance**: Compositional widgets maintain Flutter's performance characteristics—each primitive is already optimized, and const constructors can be applied to static parts.

---

## **35.2 CustomPainter and Canvas API**

When compositional widgets aren't sufficient (charts, games, signatures, complex shapes), `CustomPainter` provides direct Skia/Impeller canvas access.

### **Canvas Drawing Fundamentals**

```dart
// File: lib/custom_widgets/custom_painters.dart
import 'package:flutter/material.dart';
import 'dart:math' as math;

// Basic CustomPainter structure
class BarChartPainter extends CustomPainter {
  final List<double> data;
  final List<Color> barColors;
  
  BarChartPainter({
    required this.data,
    required this.barColors,
  });
  
  @override
  void paint(Canvas canvas, Size size) {
    // Define drawing area with padding
    final padding = 20.0;
    final drawWidth = size.width - (padding * 2);
    final drawHeight = size.height - (padding * 2);
    
    // Calculate bar dimensions
    final barWidth = (drawWidth / data.length) * 0.6;
    final spacing = (drawWidth / data.length) * 0.4;
    final maxValue = data.reduce((a, b) => a > b ? a : b);
    
    // Create reusable paint object
    final paint = Paint()
      ..strokeWidth = 2.0
      ..style = PaintingStyle.fill;
    
    // Draw each bar
    for (int i = 0; i < data.length; i++) {
      final value = data[i];
      final barHeight = (value / maxValue) * drawHeight;
      
      // Calculate position
      final left = padding + (i * (barWidth + spacing));
      final top = size.height - padding - barHeight;
      final right = left + barWidth;
      final bottom = size.height - padding;
      
      // Set color for this bar
      paint.color = barColors[i % barColors.length];
      
      // Draw rounded rectangle for bar
      final rect = RRect.fromRectAndRadius(
        Rect.fromLTRB(left, top, right, bottom),
        Radius.circular(4.0),
      );
      
      canvas.drawRRect(rect, paint);
      
      // Draw value label on top
      _drawText(
        canvas,
        value.toStringAsFixed(1),
        Offset(left + barWidth / 2, top - 15),
        size: 12,
      );
    }
    
    // Draw baseline
    final baselinePaint = Paint()
      ..color = Colors.grey
      ..strokeWidth = 1.0;
    
    canvas.drawLine(
      Offset(padding, size.height - padding),
      Offset(size.width - padding, size.height - padding),
      baselinePaint,
    );
  }
  
  void _drawText(
    Canvas canvas,
    String text,
    Offset center, {
    required double size,
    Color color = Colors.black,
  }) {
    // Using TextPainter for custom text on canvas
    final textPainter = TextPainter(
      text: TextSpan(
        text: text,
        style: TextStyle(
          color: color,
          fontSize: size,
          fontWeight: FontWeight.bold,
        ),
      ),
      textDirection: TextDirection.ltr,
      textAlign: TextAlign.center,
    );
    
    textPainter.layout();
    
    // Center text at specified position
    final offset = Offset(
      center.dx - (textPainter.width / 2),
      center.dy - (textPainter.height / 2),
    );
    
    textPainter.paint(canvas, offset);
  }
  
  @override
  bool shouldRepaint(covariant BarChartPainter oldDelegate) {
    // Only repaint if data changed
    return oldDelegate.data != data;
  }
}

// Complex painter: Circular progress with gradient
class GradientArcPainter extends CustomPainter {
  final double progress; // 0.0 to 1.0
  final double strokeWidth;
  final List<Color> gradientColors;
  
  GradientArcPainter({
    required this.progress,
    required this.strokeWidth,
    required this.gradientColors,
  });
  
  @override
  void paint(Canvas canvas, Size size) {
    final center = Offset(size.width / 2, size.height / 2);
    final radius = (size.width - strokeWidth) / 2;
    
    // Background circle (track)
    final trackPaint = Paint()
      ..color = Colors.grey[300]!
      ..style = PaintingStyle.stroke
      ..strokeWidth = strokeWidth
      ..strokeCap = StrokeCap.round;
    
    canvas.drawCircle(center, radius, trackPaint);
    
    // Progress arc with gradient
    final rect = Rect.fromCircle(center: center, radius: radius);
    
    // Create sweep gradient shader
    final gradient = SweepGradient(
      startAngle: -math.pi / 2, // Start at top
      endAngle: -math.pi / 2 + (2 * math.pi * progress),
      colors: gradientColors,
      stops: [0.0, 1.0],
      tileMode: TileMode.clamp,
    );
    
    final progressPaint = Paint()
      ..shader = gradient.createShader(rect)
      ..style = PaintingStyle.stroke
      ..strokeWidth = strokeWidth
      ..strokeCap = StrokeCap.round;
    
    // Draw arc
    canvas.drawArc(
      rect,
      -math.pi / 2, // Start angle (12 o'clock)
      2 * math.pi * progress, // Sweep angle
      false, // Don't use center (just arc, not pie)
      progressPaint,
    );
  }
  
  @override
  bool shouldRepaint(covariant GradientArcPainter oldDelegate) {
    return oldDelegate.progress != progress;
  }
}

// Interactive painter: Drawing pad
class DrawingPainter extends CustomPainter {
  final List<Offset?> points; // Null indicates line break (lifted stylus)
  final Color strokeColor;
  final double strokeWidth;
  
  DrawingPainter({
    required this.points,
    required this.strokeColor,
    required this.strokeWidth,
  });
  
  @override
  void paint(Canvas canvas, Size size) {
    final paint = Paint()
      ..color = strokeColor
      ..strokeWidth = strokeWidth
      ..strokeCap = StrokeCap.round
      ..strokeJoin = StrokeJoin.round
      ..style = PaintingStyle.stroke;
    
    // Draw continuous segments between points
    for (int i = 0; i < points.length - 1; i++) {
      final current = points[i];
      final next = points[i + 1];
      
      // Skip if either point is null (line break) or same point
      if (current != null && next != null) {
        canvas.drawLine(current, next, paint);
      }
    }
  }
  
  @override
  bool shouldRepaint(covariant DrawingPainter oldDelegate) {
    return oldDelegate.points != points;
  }
}

// Widget wrapper for drawing pad
class DrawingPad extends StatefulWidget {
  final Color strokeColor;
  final double strokeWidth;
  
  const DrawingPad({
    Key? key,
    this.strokeColor = Colors.black,
    this.strokeWidth = 5.0,
  }) : super(key: key);
  
  @override
  _DrawingPadState createState() => _DrawingPadState();
}

class _DrawingPadState extends State<DrawingPad> {
  final List<Offset?> _points = [];
  
  @override
  Widget build(BuildContext context) {
    return GestureDetector(
      onPanStart: (details) {
        setState(() {
          _points.add(details.localPosition);
        });
      },
      onPanUpdate: (details) {
        setState(() {
          _points.add(details.localPosition);
        });
      },
      onPanEnd: (_) {
        setState(() {
          _points.add(null); // Line break marker
        });
      },
      child: CustomPaint(
        size: Size.infinite,
        painter: DrawingPainter(
          points: _points,
          strokeColor: widget.strokeColor,
          strokeWidth: widget.strokeWidth,
        ),
      ),
    );
  }
  
  void clear() {
    setState(() {
      _points.clear();
    });
  }
}
```

**Explanation:**

- **Canvas coordinate system**: Origin (0,0) is top-left. X increases right, Y increases down. Angles are in radians, 0 is 3 o'clock, positive is clockwise.
- **Paint configuration**: `Paint` objects are lightweight but configure GPU state (color, stroke width, shaders). Create them once in `paint()` and reuse, or cache as instance fields.
- **TextPainter**: For text in CustomPainter, use `TextPainter` not `Text` widget. Must call `layout()` before `paint()`, and specify `textDirection`.
- **SweepGradient**: Creates angular gradients perfect for circular progress indicators. `createShader()` converts gradients to `Shader` objects attachable to Paint.
- **Performance optimization**: The `DrawingPainter` example shows efficient line drawing—instead of creating Path objects for every stroke, it draws line segments directly. For very complex drawings, Path batching is better.

---

## **35.3 CustomClipper and Clipping Paths**

Custom clippers define non-rectangular widget boundaries using Bezier curves and shapes.

### **Shape Clipping and Masking**

```dart
// File: lib/custom_widgets/custom_clippers.dart
import 'package:flutter/material.dart';
import 'dart:math' as math;

// Wave clipper for bottom sheets or headers
class WaveClipper extends CustomClipper<Path> {
  final double waveHeight;
  final double waveCount;
  
  WaveClipper({
    this.waveHeight = 50.0,
    this.waveCount = 2.0,
  });
  
  @override
  Path getClip(Size size) {
    final path = Path();
    
    // Start at top-left
    path.lineTo(0, size.height - waveHeight);
    
    // Create wave using Bezier curves
    final waveWidth = size.width / waveCount;
    
    for (int i = 0; i < waveCount; i++) {
      final startX = i * waveWidth;
      final endX = (i + 1) * waveWidth;
      final midX = startX + (waveWidth / 2);
      
      // Control points for cubic bezier
      final controlPoint1 = Offset(midX, size.height + waveHeight);
      final controlPoint2 = Offset(midX, size.height - waveHeight);
      
      path.cubicTo(
        controlPoint1.dx, controlPoint1.dy, // CP1
        controlPoint2.dx, controlPoint2.dy, // CP2
        endX, size.height - waveHeight,     // End point
      );
    }
    
    // Complete the path
    path.lineTo(size.width, 0);
    path.lineTo(0, 0);
    path.close();
    
    return path;
  }
  
  @override
  bool shouldReclip(covariant WaveClipper oldClipper) {
    return oldClipper.waveHeight != waveHeight ||
           oldCliper.waveCount != waveCount;
  }
}

// Usage
class WaveHeader extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return ClipPath(
      clipper: WaveClipper(waveHeight: 80, waveCount: 3),
      child: Container(
        height: 250,
        color: Colors.blue,
        child: Center(
          child: Text(
            'Wave Design',
            style: TextStyle(
              color: Colors.white,
              fontSize: 32,
              fontWeight: FontWeight.bold,
            ),
          ),
        ),
      ),
    );
  }
}

// Hexagonal clipper for profile avatars
class HexagonClipper extends CustomClipper<Path> {
  @override
  Path getClip(Size size) {
    final path = Path();
    final center = Offset(size.width / 2, size.height / 2);
    final radius = size.width / 2;
    
    // Create hexagon (6 sides)
    for (int i = 0; i < 6; i++) {
      final angle = (i * 60) * (math.pi / 180); // Convert to radians
      final x = center.dx + radius * math.cos(angle);
      final y = center.dy + radius * math.sin(angle);
      
      if (i == 0) {
        path.moveTo(x, y);
      } else {
        path.lineTo(x, y);
      }
    }
    
    path.close();
    return path;
  }
  
  @override
  bool shouldReclip(covariant CustomClipper<Path> oldClipper) => false;
}

// Star shape clipper
class StarClipper extends CustomClipper<Path> {
  final int points;
  final double innerRadiusRatio;
  
  StarClipper({
    this.points = 5,
    this.innerRadiusRatio = 0.5,
  });
  
  @override
  Path getClip(Size size) {
    final path = Path();
    final center = Offset(size.width / 2, size.height / 2);
    final outerRadius = size.width / 2;
    final innerRadius = outerRadius * innerRadiusRatio;
    
    final angleStep = math.pi / points; // Half circle per point
    
    for (int i = 0; i < points * 2; i++) {
      final radius = i.isEven ? outerRadius : innerRadius;
      final angle = (i * angleStep) - (math.pi / 2); // Start at top
      
      final x = center.dx + radius * math.cos(angle);
      final y = center.dy + radius * math.sin(angle);
      
      if (i == 0) {
        path.moveTo(x, y);
      } else {
        path.lineTo(x, y);
      }
    }
    
    path.close();
    return path;
  }
  
  @override
  bool shouldReclip(covariant StarClipper oldClipper) {
    return oldClipper.points != points ||
           oldClipper.innerRadiusRatio != innerRadiusRatio;
  }
}

// Complex: Ticket clipper with circular notches
class TicketClipper extends CustomClipper<Path> {
  final double notchRadius;
  
  TicketClipper({this.notchRadius = 20.0});
  
  @override
  Path getClip(Size size) {
    final path = Path();
    
    // Top edge with two notches
    path.moveTo(0, 0);
    path.lineTo(size.width * 0.3 - notchRadius, 0);
    
    // First notch (semicircle cut out)
    path.arcToPoint(
      Offset(size.width * 0.3 + notchRadius, 0),
      radius: Radius.circular(notchRadius),
      clockwise: false, // Cut outward
    );
    
    path.lineTo(size.width * 0.7 - notchRadius, 0);
    
    // Second notch
    path.arcToPoint(
      Offset(size.width * 0.7 + notchRadius, 0),
      radius: Radius.circular(notchRadius),
      clockwise: false,
    );
    
    path.lineTo(size.width, 0);
    path.lineTo(size.width, size.height);
    path.lineTo(0, size.height);
    path.close();
    
    return path;
  }
  
  @override
  bool shouldReclip(covariant CustomClipper<Path> oldClipper) => false;
}

// Gradient mask using ShaderMask
class GradientMask extends StatelessWidget {
  final Widget child;
  
  const GradientMask({Key? key, required this.child}) : super(key: key);
  
  @override
  Widget build(BuildContext context) {
    return ShaderMask(
      // ShaderMask applies a shader as an alpha mask
      shaderCallback: (bounds) {
        return LinearGradient(
          begin: Alignment.topCenter,
          end: Alignment.bottomCenter,
          colors: [
            Colors.white, // Opaque
            Colors.white,
            Colors.transparent, // Fade out
          ],
          stops: [0.0, 0.7, 1.0],
        ).createShader(bounds);
      },
      blendMode: BlendMode.dstIn, // Use shader alpha to mask child
      child: child,
    );
  }
}
```

**Explanation:**

- **CustomClipper<Path>**: Returns a `Path` object defining the visible region. Everything outside the path is clipped (invisible).
- **Bezier curves**: `cubicTo()` creates smooth curves using two control points. For waves, control points extend above and below the baseline to create S-curves.
- **Arc construction**: `arcToPoint()` draws circular arcs between points, useful for rounded corners or notches (like the ticket example).
- **ShaderMask**: Unlike `ClipPath` which is binary (inside/outside), `ShaderMask` uses gradients to create alpha fades. `BlendMode.dstIn` means "keep destination (child) where source (shader) is opaque."
- **shouldReclip**: Return `true` only when parameters change. The ticket clipper returns `false` because it has no configurable parameters—Flutter can reuse the clip path.

---

## **35.4 Gradient Shaders and Visual Effects**

Advanced Paint configurations enable gradients, patterns, and blend modes for sophisticated visuals.

### **Shader Effects**

```dart
// File: lib/custom_widgets/shader_effects.dart
import 'package:flutter/material.dart';
import 'dart:math' as math;

// Sweep gradient clock face
class GradientClockPainter extends CustomPainter {
  final DateTime time;
  
  GradientClockPainter(this.time);
  
  @override
  void paint(Canvas canvas, Size size) {
    final center = Offset(size.width / 2, size.height / 2);
    final radius = size.width / 2;
    
    // Radial gradient background
    final bgPaint = Paint()
      ..shader = RadialGradient(
        colors: [Colors.blue[100]!, Colors.blue[800]!],
        stops: [0.2, 1.0],
      ).createShader(
        Rect.fromCircle(center: center, radius: radius),
      );
    
    canvas.drawCircle(center, radius, bgPaint);
    
    // Hour markers with sweep gradient
    for (int i = 0; i < 12; i++) {
      final angle = (i * 30) * (math.pi / 180) - (math.pi / 2);
      final markerStart = center + Offset(
        math.cos(angle) * (radius * 0.85),
        math.sin(angle) * (radius * 0.85),
      );
      final markerEnd = center + Offset(
        math.cos(angle) * (radius * 0.95),
        math.sin(angle) * (radius * 0.95),
      );
      
      final markerPaint = Paint()
        ..strokeWidth = 4
        ..strokeCap = StrokeCap.round
        ..color = Colors.white;
      
      canvas.drawLine(markerStart, markerEnd, markerPaint);
    }
    
    // Hands with shadows
    final shadowPaint = Paint()
      ..color = Colors.black.withOpacity(0.3)
      ..maskFilter = MaskFilter.blur(BlurStyle.normal, 4);
    
    _drawHand(
      canvas,
      center,
      time.hour * 30 + time.minute * 0.5 - 90,
      radius * 0.5,
      6,
      Colors.white,
      shadowPaint,
    );
    
    _drawHand(
      canvas,
      center,
      time.minute * 6 + time.second * 0.1 - 90,
      radius * 0.75,
      4,
      Colors.white,
      shadowPaint,
    );
    
    // Center dot
    canvas.drawCircle(
      center,
      8,
      Paint()..color = Colors.white,
    );
  }
  
  void _drawHand(
    Canvas canvas,
    Offset center,
    double degrees,
    double length,
    double width,
    Color color,
    Paint shadowPaint,
  ) {
    final angle = degrees * (math.pi / 180);
    final end = center + Offset(
      math.cos(angle) * length,
      math.sin(angle) * length,
    );
    
    // Draw shadow first (offset slightly)
    final shadowEnd = end + Offset(2, 2);
    canvas.drawLine(center + Offset(2, 2), shadowEnd, shadowPaint);
    
    // Draw hand
    final handPaint = Paint()
      ..color = color
      ..strokeWidth = width
      ..strokeCap = StrokeCap.round;
    
    canvas.drawLine(center, end, handPaint);
  }
  
  @override
  bool shouldRepaint(covariant GradientClockPainter oldDelegate) {
    return oldDelegate.time != time;
  }
}

// Pattern painting with tiling
class PatternPainter extends CustomPainter {
  @override
  void paint(Canvas canvas, Size size) {
    // Create a small pattern image
    final patternPaint = Paint()
      ..shader = ImageShader(
        // Normally you'd load an asset image here
        // For demo, using gradient as pattern substitute
        createPatternImage(),
        TileMode.repeated,
        TileMode.repeated,
        Matrix4.identity().storage,
      );
    
    canvas.drawRect(
      Offset.zero & size,
      patternPaint,
    );
  }
  
  // Helper to create pattern (in real app, load from assets)
  ui.Image createPatternImage() {
    // Implementation would create/return a small repeating texture
    throw UnimplementedError();
  }
  
  @override
  bool shouldRepaint(covariant CustomPainter oldDelegate) => false;
}

// Blend modes for visual effects
class BlendModeDemo extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return Stack(
      children: [
        // Base image
        Image.network('https://example.com/photo.jpg'),
        
        // Overlay with blend mode
        CustomPaint(
          size: Size.infinite,
          painter: VignettePainter(),
        ),
      ],
    );
  }
}

class VignettePainter extends CustomPainter {
  @override
  void paint(Canvas canvas, Size size) {
    final rect = Offset.zero & size;
    
    // Create radial gradient for vignette (dark edges)
    final gradient = RadialGradient(
      center: Alignment.center,
      radius: 0.8,
      colors: [
        Colors.transparent,
        Colors.black.withOpacity(0.7),
      ],
      stops: [0.5, 1.0],
    );
    
    final paint = Paint()
      ..shader = gradient.createShader(rect)
      ..blendMode = BlendMode.multiply; // Darken underlying image
    
    canvas.drawRect(rect, paint);
  }
  
  @override
  bool shouldRepaint(covariant CustomPainter oldDelegate) => false;
}

// Water ripple effect using radial gradient animation
class RipplePainter extends CustomPainter {
  final double progress; // 0.0 to 1.0
  final Offset center;
  final Color color;
  
  RipplePainter({
    required this.progress,
    required this.center,
    required this.color,
  });
  
  @override
  void paint(Canvas canvas, Size size) {
    final maxRadius = math.sqrt(
      math.pow(size.width, 2) + math.pow(size.height, 2),
    );
    
    final currentRadius = maxRadius * progress;
    
    // Create ripple with fading alpha
    final paint = Paint()
      ..style = PaintingStyle.stroke
      ..strokeWidth = 4.0 * (1 - progress) // Get thinner as it expands
      ..color = color.withOpacity(1 - progress); // Fade out
    
    canvas.drawCircle(center, currentRadius, paint);
    
    // Multiple rings for echo effect
    if (progress > 0.3) {
      final innerPaint = Paint()
        ..style = PaintingStyle.stroke
        ..strokeWidth = 2.0
        ..color = color.withOpacity(0.5 * (1 - progress));
      
      canvas.drawCircle(center, currentRadius * 0.7, innerPaint);
    }
  }
  
  @override
  bool shouldRepaint(covariant RipplePainter oldDelegate) {
    return oldDelegate.progress != progress;
  }
}
```

**Explanation:**

- **RadialGradient**: Creates color transitions emanating from a center point. Perfect for clock faces, spotlights, and vignettes. Use `createShader()` with a rectangle defining the gradient bounds.
- **MaskFilter.blur**: Applies Gaussian blur to Paint operations. Used for drop shadows behind clock hands. Note: Blurs are GPU-intensive; use sparingly.
- **BlendMode**: Controls how painted pixels combine with existing canvas content. `BlendMode.multiply` darkens underlying content (useful for vignettes/overlays). Other useful modes: `screen`, `overlay`, `colorBurn`.
- **ImageShader**: Allows tiling small texture images across large areas. `TileMode.repeated` creates seamless patterns.
- **Animation support**: `RipplePainter` demonstrates animated custom painting—`progress` parameter drives the animation state, and `shouldRepaint` returns `true` when progress changes.

---

## **35.5 Implicitly Animated Widgets**

Flutter provides built-in support for animating between property values without managing AnimationControllers manually.

### **Built-in Animated Widgets**

```dart
// File: lib/custom_widgets/implicit_animations.dart
import 'package:flutter/material.dart';

// AnimatedContainer - smooth transitions between box properties
class AnimatedCard extends StatefulWidget {
  @override
  _AnimatedCardState createState() => _AnimatedCardState();
}

class _AnimatedCardState extends State<AnimatedCard> {
  bool _isExpanded = false;
  
  @override
  Widget build(BuildContext context) {
    return GestureDetector(
      onTap: () => setState(() => _isExpanded = !_isExpanded),
      child: AnimatedContainer(
        duration: Duration(milliseconds: 300),
        curve: Curves.easeInOut,
        
        // Animated properties:
        width: _isExpanded ? 300 : 150,
        height: _isExpanded ? 400 : 200,
        decoration: BoxDecoration(
          color: _isExpanded ? Colors.blue[700] : Colors.blue[400],
          borderRadius: BorderRadius.circular(_isExpanded ? 20 : 10),
          boxShadow: [
            BoxShadow(
              color: Colors.black26,
              blurRadius: _isExpanded ? 20 : 10,
              offset: Offset(0, _isExpanded ? 10 : 5),
            ),
          ],
        ),
        
        child: Center(
          child: Text(
            'Tap Me',
            style: TextStyle(color: Colors.white),
          ),
        ),
      ),
    );
  }
}

// TweenAnimationBuilder - animate any property with custom builder
class AnimatedCounter extends StatelessWidget {
  final int value;
  
  const AnimatedCounter({Key? key, required this.value}) : super(key: key);
  
  @override
  Widget build(BuildContext context) {
    return TweenAnimationBuilder<int>(
      tween: IntTween(begin: 0, end: value),
      duration: Duration(seconds: 1),
      curve: Curves.easeOutCubic,
      builder: (context, animatedValue, child) {
        return Text(
          '$animatedValue',
          style: TextStyle(fontSize: 48, fontWeight: FontWeight.bold),
        );
      },
    );
  }
}

// AnimatedPositioned - smooth layout changes in Stack
class AnimatedReorderDemo extends StatefulWidget {
  @override
  _AnimatedReorderDemoState createState() => _AnimatedReorderDemoState();
}

class _AnimatedReorderDemoState extends State<AnimatedReorderDemo> {
  List<Color> colors = [Colors.red, Colors.green, Colors.blue, Colors.yellow];
  
  void _shuffle() {
    setState(() {
      colors.shuffle();
    });
  }
  
  @override
  Widget build(BuildContext context) {
    return Stack(
      children: colors.asMap().entries.map((entry) {
        final index = entry.key;
        final color = entry.value;
        
        return AnimatedPositioned(
          duration: Duration(milliseconds: 500),
          curve: Curves.elasticOut,
          left: (index % 2) * 150.0,
          top: (index ~/ 2) * 150.0,
          width: 120,
          height: 120,
          child: GestureDetector(
            onTap: _shuffle,
            child: Container(
              color: color,
              child: Center(child: Text('${index + 1}')),
            ),
          ),
        );
      }).toList(),
    );
  }
}

// Custom implicit animation: AnimatedOpacity with custom widget
class FadeInImage extends StatefulWidget {
  final String imageUrl;
  
  const FadeInImage({Key? key, required this.imageUrl}) : super(key: key);
  
  @override
  _FadeInImageState createState() => _FadeInImageState();
}

class _FadeInImageState extends State<FadeInImage> 
    with SingleTickerProviderStateMixin {
  late AnimationController _controller;
  
  @override
  void initState() {
    super.initState();
    _controller = AnimationController(
      duration: Duration(milliseconds: 500),
      vsync: this,
    );
    
    // Load image then fade in
    _loadImage();
  }
  
  Future<void> _loadImage() async {
    // Simulate network loading
    await Future.delayed(Duration(seconds: 1));
    if (mounted) _controller.forward();
  }
  
  @override
  Widget build(BuildContext context) {
    return FadeTransition(
      opacity: _controller,
      child: Image.network(widget.imageUrl),
    );
  }
  
  @override
  void dispose() {
    _controller.dispose();
    super.dispose();
  }
}

// Hero animations for page transitions
class HeroAnimationDemo extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      body: GridView.builder(
        gridDelegate: SliverGridDelegateWithFixedCrossAxisCount(
          crossAxisCount: 2,
        ),
        itemCount: 10,
        itemBuilder: (context, index) {
          return GestureDetector(
            onTap: () {
              Navigator.push(
                context,
                MaterialPageRoute(
                  builder: (context) => DetailPage(index: index),
                ),
              );
            },
            child: Hero(
              // Unique tag identifies the shared element
              tag: 'image_$index',
              child: Container(
                margin: EdgeInsets.all(8),
                color: Colors.primaries[index % Colors.primaries.length],
                child: Center(child: Text('Item $index')),
              ),
            ),
          );
        },
      ),
    );
  }
}

class DetailPage extends StatelessWidget {
  final int index;
  
  const DetailPage({Key? key, required this.index}) : super(key: key);
  
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(),
      body: Center(
        child: Hero(
          tag: 'image_$index',
          child: Container(
            width: 300,
            height: 300,
            color: Colors.primaries[index % Colors.primaries.length],
            child: Center(
              child: Text(
                'Detail $index',
                style: TextStyle(fontSize: 32, color: Colors.white),
              ),
            ),
          ),
        ),
      ),
    );
  }
}
```

**Explanation:**

- **AnimatedContainer**: Implicitly animates between old and new values of decoration, size, padding, etc. Just change properties in `setState()`—Flutter handles the animation interpolation.
- **TweenAnimationBuilder**: Lower-level implicit animation. Define a `Tween` (value range), duration, and builder function. Flutter calls the builder with interpolated values during the animation.
- **AnimatedPositioned**: Specialized for Stack children. Changing `left`/`top`/`width`/`height` triggers automatic animation to new layout position.
- **Hero**: Creates shared element transitions between routes. Widgets with matching `tag` values animate from source to destination route automatically during navigation.
- **Performance**: Implicit animations use `AnimationController` internally but manage disposal automatically. They're less flexible than explicit controllers but require much less code for simple transitions.

---

## **Chapter Summary**

In this chapter, we explored creating custom visual components in Flutter:

### **Key Takeaways:**

1. **Composite Widgets**: Prefer composing existing widgets before custom painting. Use `Container` with `BoxDecoration`, `ClipRRect`, and `Stack` for complex layouts without custom render objects.

2. **CustomPainter**: For charts, signatures, and game graphics, extend `CustomPainter`. Cache `Paint` objects, use `Path` for shapes, and implement `shouldRepaint` to avoid unnecessary redraws.

3. **CustomClipper**: Create non-rectangular widget shapes using `Path` with lines, arcs, and Bezier curves. `CustomClipper<Path>` defines visible regions; `ShaderMask` creates alpha fades.

4. **Shaders and Effects**: Use `Gradient.createShader()` for advanced color transitions. Apply `MaskFilter.blur` for shadows and `BlendMode` for overlay effects like vignettes.

5. **Implicit Animations**: Use `AnimatedContainer`, `AnimatedPositioned`, and `TweenAnimationBuilder` for automatic transitions between property values without managing controllers.

6. **When to Custom Paint**: Use CustomPainter when you need: real-time drawing (signatures), data visualization (thousands of data points), game graphics, or effects impossible with widget composition.

### **Next Steps:**

Chapter 36 will cover **Animations Mastery** in depth:
- AnimationController and TickerProvider deep dive
- CurvedAnimation and easing functions
- Hero animations and shared element transitions
- Staggered animations and sequence control
- Lottie integration and Rive animations
- Physics-based animations (springs, gravity)

---

**End of Chapter 35**

---

# **Next Chapter: Chapter 36 - Animations Mastery**

Chapter 36 will explore advanced animation techniques, physics simulations, and integration with external animation tools like Lottie and Rive.

